In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # Environment setup
# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Rapid API Try

In [ ]:
text="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each breath. Let your thoughts come and go like clouds in the sky. No need to follow them or push them away. Just return to the rhythm of your breath—anchoring yourself in the present moment. You are safe. You are calm. You are grounded. Inhale deeply through your nose… and exhale gently through your mouth. With each breath, feel a sense of peace expanding within you. Now, bring to mind a feeling of gratitude. It could be for a person, a place, or even just the gift of this moment. Let that gratitude warm your heart. Take one more deep breath in… and sigh it out. As we near the end of this meditation, slowly bring your awareness back to your surroundings. Wiggle your fingers and toes. Gently roll your shoulders. When you're ready, open your eyes. Take a moment to notice how you feel. Carry this calm and clarity with you into the rest of your day. Thank you for taking this time for yourself."

In [ ]:
import requests

url = "https://text-to-speach-api.p.rapidapi.com/text-to-speech"

payload = {
	"text": text,
	"lang": "en",
	"speed": "medium"
}
headers = {
	"x-rapidapi-key": "key",
	"x-rapidapi-host": "text-to-speach-api.p.rapidapi.com",
	"Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'error': 'Text must not exceed 200 characters.'}


In [ ]:
from pathlib import Path
from openai import OpenAI
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_TK')


# client = OpenAI()
# speech_file_path = Path(__file__).parent / "speech.mp3"

# with client.audio.speech.with_streaming_response.create(
#     model="gpt-4o-mini-tts",
#     voice="coral",
#     input="Today is a wonderful day to build something people love!",
#     instructions="Speak in a cheerful and positive tone.",
# ) as response:
#     response.stream_to_file(speech_file_path)

# TTS Model Estimation

In [ ]:
from pathlib import Path
from openai import OpenAI
import tiktoken
import os
from pydub import AudioSegment

# --- Constants for pricing ---
CHAR_COST_PER_MILLION = 12.00  # $12 per 1M characters
MINIMUM_COST = 0.60  # flat fee per request (if applicable)

# --- Setup
client = OpenAI()
speech_file_path = "/content/speech.mp3"

# --- Input
input_text = "Today is a wonderful day to build something people love!"

# --- Estimate input tokens and characters
encoding = tiktoken.get_encoding("cl100k_base")  # Used by GPT-4o
input_tokens = len(encoding.encode(input_text))
input_chars = len(input_text)

# --- Generate speech and save to file
with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input=input_text,
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

# --- Output: File size and duration
output_size_bytes = os.path.getsize(speech_file_path)
audio = AudioSegment.from_file(speech_file_path)
duration_seconds = len(audio) / 1000

# --- Cost Calculation
char_cost = (input_chars / 1_000_000) * CHAR_COST_PER_MILLION
total_cost = max(char_cost, MINIMUM_COST)  # Assume flat minimum applies

# --- Display Info
print(f"Input text: {input_text}")
print(f"Input characters: {input_chars}")
print(f"Input tokens (estimated): {input_tokens}")
print(f"Audio duration: {duration_seconds:.2f} seconds")
print(f"Output file size: {output_size_bytes / 1024:.2f} KB")
print(f"Estimated TTS cost: ${total_cost:.4f}")


Input text: Today is a wonderful day to build something people love!
Input characters: 56
Input tokens (estimated): 11
Audio duration: 4.10 seconds
Output file size: 64.12 KB
Estimated TTS cost: $0.6000
